The **Part 1** of the project is divided into three sections:

1- Feature Extraction (Using SIFT)

2- Outlier Removal (Using RANSAC)

3- Computing the Homographies (Using DLT)


**pip install opencv-python**

**pip install opencv-contrib-python**

**Part 1**

In [15]:
#Imports
from numpy.linalg import eig
import numpy as np
import cv2
import os

In [35]:
#Display the video
capture = cv2.VideoCapture(os.path.abspath('Video/trymefirst_lisbon.mp4')) 
while(capture.isOpened()):
    ret, frame = capture.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  ##press q if you want the video to stop 
        break
capture.release()
cv2.destroyAllWindows()

The following code **extracts SIFT features** from each frame of the input video

In [16]:

capture = cv2.VideoCapture(os.path.abspath('Video/trymefirst_lisbon.mp4'))
kp_list = []
sift_points = [] #nome a definir no config
t = 0 
sift = cv2.SIFT_create(5000) #number of sift points
while True:
    t = t + 1
    if t == 5: break 
    success, frame = capture.read() #read the video
    if success:
        frame_points = []
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) #convert image to gray
        kp, des = sift.detectAndCompute(gray,None) #kp = keypoint, des = descriptor
        kp_list.append(kp)
        frame_points = ([kp[0].pt[0],kp[0].pt[1]]+des[0].tolist())
        for i in range(1,len(kp)):
             list = ([kp[i].pt[0],kp[i].pt[1]]+des[i].tolist())
             frame_points = np.column_stack((frame_points,list))  
        sift_points.append(frame_points) #append everything into a list 
#print(des.shape)
#print(len(sift_points))
#The keypoint is a point of interest in the image, the descriptor is a vector that describes the image patch around the keypoint

The following code **matches SIFT features** between the frames

In [17]:
#Brute force method
bf = cv2.BFMatcher(crossCheck=True) #crossCheck is set to true so that the match is symmetric
all_matches = []
match = []
for s in range(len(sift_points)-1):
    point_matches = []

    des1 = (((sift_points[s])[2:,:])).astype('float32')  # descriptors of the first frame
    des2 = (((sift_points[s+1])[2:,:])).astype('float32')  # descriptors of the second
    des1 = np.reshape(des1,(np.shape(des1)[1],128))
    des2 = np.reshape(des2,(np.shape(des2)[1],128))

    if np.shape(des1)[0] > np.shape(des2)[0]:
             des1 = des1[:-abs(np.shape(des1)[0]-np.shape(des2)[0]),:]  # we are removing the last points so that we have an equal amount of SIFT features between two frames
    if np.shape(des1)[0] < np.shape(des2)[0]:
             des2 = des2[:-abs(np.shape(des1)[0]-np.shape(des2)[0]),:]
    matches = bf.match(des1,des2)  # an error occurs if two frames have different amounts of SIFT features
    # try:
    # except: 
    #       print('Error!')        
             
    for i in range(len(matches)):
        match.append(matches)
        point_matches.append([matches[i].queryIdx,matches[i].trainIdx])

    all_matches.append(point_matches)
#Feature detection: opencv
#Matching : sklearn , numpy
#RANSAC: numpy
#Create Homography: numpy

The following code **computes the Homography** between the frames of the video

In [18]:
import numpy as np

def normalize(points):
    mean = np.mean(points, axis=0)
    std_dev = np.std(points)
    T = np.array([[std_dev, 0, mean[0]], [0, std_dev, mean[1]], [0, 0, 1]])
    T_inv = np.linalg.inv(T)
    normalized_points = np.dot(T_inv, np.append(points, np.ones((points.shape[0], 1)), axis=1).T).T
    return normalized_points, T

def construct_matrix_A(points1, points2):
    A = []
    for i in range(points1.shape[0]):
        x1, y1 = points1[i, 0], points1[i, 1]
        x2, y2 = points2[i, 0], points2[i, 1]
        A.append([-x1, -y1, -1, 0, 0, 0, x2*x1, x2*y1, x2])
        A.append([0, 0, 0, -x1, -y1, -1, y2*x1, y2*y1, y2])
    return np.array(A)

def compute_homography(points1, points2):
    points1_norm, T1 = normalize(points1)
    points2_norm, T2 = normalize(points2)
    A = construct_matrix_A(points1_norm, points2_norm)
    _, _, V = np.linalg.svd(A)
    H = V[-1].reshape(3, 3)
    H = np.dot(np.linalg.inv(T2), np.dot(H, T1))
    return H / H[2, 2]


In [56]:
def normalize_points(points):
    # Normalize points to have zero mean and unit variance
    mean = np.mean(points, axis=0)
    std = np.std(points, axis=0)
    normalized_points = (points - mean) / std
    return normalized_points, mean, std


def denormalize_homography(H, mean_src, std_src, mean_dst, std_dst):
    # Denormalize the homography matrix based on mean and standard deviation
   # T_src = np.array([[1 / std_src[0], 0, -mean_src[0] / std_src[0]],
            #          [0, 1 / std_src[1], -mean_src[1] / std_src[1]],
            #          [0, 0, 1]])

   # T_dst = np.array([[1 / std_dst[0], 0, -mean_dst[0] / std_dst[0]],
   #                   [0, 1 / std_dst[1], -mean_dst[1] / std_dst[1]],
   #                   [0, 0, 1]])
    T_src = np.array([[std_src[0], 0, mean_src[0]], [0, std_src[1], mean_src[1]], [0, 0, 1]])
    T_dst = np.array([[std_dst[0], 0, mean_dst[0]], [0, std_src[1], mean_dst[1]], [0, 0, 1]])

    Homography = np.dot(np.linalg.inv(T_dst), np.dot(H, T_src))
    
    return  Homography



In [57]:
from sklearn import preprocessing
kp1 = kp_list[0]
kp2 = kp_list[1]
src_pts = np.float32([ kp1[q.queryIdx].pt for q in match[0] ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[t.trainIdx].pt for t in match[1] ]).reshape(-1,1,2)
src = np.reshape(src_pts,(np.shape(src_pts)[0],2))
dst = np.reshape(dst_pts,(np.shape(dst_pts)[0],2))


src_pts_normalized, mean_src, std_src = normalize_points(src)
dst_pts_normalized, mean_dst, std_dst = normalize_points(dst)
#src = preprocessing.normalize(src)   #Normalization
#dst = preprocessing.normalize(dst)


In [82]:
def Comp_H(src,dst):
        A = []
        for p, q in zip(src, dst):
            x1 = p[0]
            y1 = p[1]
            x2 = q[0]
            y2 = q[1]
            A.append([-x1, -y1, -1, 0, 0, 0, x2*x1, x2*y1, x2])
            A.append([0, 0, 0, -x1, -y1, -1, y2*x1, y2*y1, y2])

        _, _, Vt = np.linalg.svd(A, full_matrices=True)
        x = Vt[-1]
        homography = x.reshape(3, -1) / x[-1]
        return homography

def RANSAC(Comp_H,src,dst,iter,threshold):
      best_homography = None
      inliers = [0]
      for t in range(iter):
            sample_indices = np.random.choice(int(len(src)), size=4, replace=False)
            # Compute the Homography
            H = Comp_H(src[sample_indices],dst[sample_indices])
            #H = denormalize_homography(homography, mean_src, std_src, mean_dst, std_dst)
            inl = 0
            for p, q in zip(src, dst):
                x1 = p[0]
                y1 = p[1]
                x2 = q[0]
                y2 = q[1]
            # Transform the point using the estimated homography
                transformed_point = np.dot(H, np.array([x1, y1, 1]))

            # Normalize the transformed point
                transformed_point /= transformed_point[2]

            # Calculate the Euclidean distance between the transformed point and the actual point
                distance = np.linalg.norm(np.array([x2, y2, 1]) - transformed_point)
                if distance < threshold:
                   inl += 1
            if inl > inliers[0]:
                 best_homography = H
                 inliers[0] = inl
      return best_homography, inliers[0] 
      

In [97]:
H, inliers = RANSAC(Comp_H,src,dst,143,30)
print('condition:',np.linalg.cond(H), 'inliers: ', inliers )
picture2 = np.reshape(np.array([552,59,1]),(3,1))
picture1 = np.reshape(np.array([549,56,1]),(3,1))
pic_h = np.matmul(H,picture1)
print('the error is: \n', pic_h/pic_h[2]-picture1)

condition: 1733.4008792564298 inliers:  60
the error is: 
 [[-217.55780203]
 [ 129.59353309]
 [   0.        ]]


**Testing Zone**

In [126]:
#src_pts = np.float32([ kp1[all_matches[0][i][0]].pt for i in range(len(all_matches[0])) ]).reshape(-1,1,2)
#dst_pts = np.float32([ kp2[all_matches[0][i][1]].pt for i in range(len(all_matches[0])) ]).reshape(-1,1,2)
#M2, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
#np.linalg.cond(M2)

kp1 = kp_list[0]
kp2 = kp_list[1]
src_pts = np.float32([ kp1[q.queryIdx].pt for q in match[0] ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[t.trainIdx].pt for t in match[1] ]).reshape(-1,1,2)
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
np.linalg.cond(M)

1.1183369800502505

In [117]:
pic_h2 = np.matmul(M,picture2)
pic_h2/pic_h2[2]

array([[-286.19830942],
       [ -61.0945108 ],
       [   1.        ]])

In [63]:
def getPerspectiveTransform(src, dst):
    if len(src) == len(dst):
        # Make homogeneous coordiates if necessary
        if src.shape[1] == 2:
            src = np.hstack((src, np.ones((len(src), 1), dtype=src.dtype)))
        if dst.shape[1] == 2:
            dst = np.hstack((dst, np.ones((len(dst), 1), dtype=dst.dtype)))

        # Solve 'Ax = 0'
        A = []
        for p, q in zip(src, dst):
            A.append([0, 0, 0, q[2]*p[0], q[2]*p[1], q[2]*p[2], -q[1]*p[0], -q[1]*p[1], -q[1]*p[2]])
            A.append([q[2]*p[0], q[2]*p[1], q[2]*p[2], 0, 0, 0, -q[0]*p[0], -q[0]*p[1], -q[0]*p[2]])

        eigenvalue,eigenvector=eig(np.matmul(np.transpose(A),A))
        _, _, Vt = np.linalg.svd(A, full_matrices=True)
        x = Vt[-1]
         

        # Reorganize `x` as a matrix
        H = x.reshape(3, -1) / x[-1] # Normalize the last element as 1
        return H
    

H_slides = getPerspectiveTransform(np.reshape(src_pts,(np.shape(src_pts)[0],2)), np.reshape(dst_pts,(np.shape(dst_pts)[0],2)))
np.linalg.cond(H_slides)

67711165.49974936

In [64]:
picture2 = np.reshape(np.array([552,59,1]),(3,1))
picture1 = np.reshape(np.array([549,56,1]),(3,1))
pic_h = np.matmul(H_slides,picture1)
pic_h/pic_h[2]

array([[492.63500599],
       [293.28418155],
       [  1.        ]])

In [219]:
pic_h3 = np.matmul(H_slides,picture2)
pic_h3/pic_h3[2]

array([[492.55113247],
       [293.19723376],
       [  1.        ]])

In [117]:
def select_point(event,x,y,flags,param):
    global ix,iy
    if event == cv2.EVENT_LBUTTONDBLCLK: # captures left button double-click
        print('x = %d, y = %d'%(x, y))


In [ ]:
capture = cv2.VideoCapture(os.path.abspath('trymefirst_lisbon.mp4'))
framenr = 0 
list_points = []
while True:
    success, frame = capture.read()
    if success:
        print('Current Frame!')
        cv2.namedWindow('frame')
        cv2.setMouseCallback('frame', select_point)
        cv2.imshow('frame',frame)
        if cv2.waitKey(0) & 0xFF == ord('q'):  ##press q if you want the video to stop 
             break
        key = cv2.waitKey(0) & 0xFF == ord('k')
        
        print('New Frame!')

capture.release()
cv2.destroyAllWindows()